### 一共会涉及到如下的几个类

#### DataSet
#### Sampler
#### Batch
#### Model
#### Loss
#### Metric
#### Trainer
#### Tester

### 下面具体讲一下它们的作用

#### DataSet: 用于承载数据。
(1) DataSet里面每个元素只能是以下的三类np.float64, np.int64, np.str。如果传入的数据是int则被转换为np.int64, float被转为np.float64。   
(2) DataSet可以将field设置为input，target。其中被设置为input的field会被传递给Model.forward, 这个过程中我们是通过键匹配完成传递的。举例来说，假设DataSet中有'x1', 'x2', 'x3'被设置为了input，而  
&emsp;&emsp;&emsp;(2.1)函数是Model.forward(self, x1, x3), 那么DataSet中'x1', 'x3'会被传递给forward函数。多余的'x2'会被忽略  
&emsp;&emsp;&emsp;(2.2)函数是Model.forward(self, x1, x4), 这里多需要了一个'x4', 但是DataSet的input field中没有这个field，会报错。  
&emsp;&emsp;&emsp;(2.3)函数是Model.forward(self, x1, **kwargs), 会把'x1', 'x2', 'x3'都传入。但如果是Model.forward(self, x4, **kwargs)就会发生报错，因为没有'x4'。  
(3) 对于设置为target的field的名称，我们建议取名为'target'（如果只有一个需要predict的值），但是不强制。后面会讲为什么target可以不强制。  
DataSet应该是不需要单独再开发的，如果有不能满足的场景，请在开发群提出或者github提交issue。

#### Sampler: 给定一个DataSet，返回一个序号的list，Batch按照这个list输出数据。
Sampler需要继承fastNLP.core.sampler.BaseSampler

#### Batch: 将DataSet中设置为input和target的field取出来构成batch_x, batch_y
并且根据情况(主要根据数据类型能不能转为Tensor)将数据转换为pytorch的Tensor。batch中sample的取出顺序是由Sampler决定的。  
Sampler是传入一个DataSet，返回一个与DataSet等长的序号list，Batch一次会取出batch_size个sample(最后一个batch可能数量不足batch_size个)。   
举例：  
(1) SequentialSampler是顺序采样
    假设传入的DataSet长度是100, SequentialSampler返回的序号list就是[0, 1, ...,98, 99]. batch_size如果被设置为4，那么第一个batch所获取的instance就是[0, 1, 2, 3]这四个instance. 第二个batch所获取instace就是[4, 5, 6, 7], ...直到采完所有的sample。   
(2) RandomSampler是随机采样   
    假设传入的DataSet长度是100, RandomSampler返回的序号list可能是[0, 99, 20, 5, 3, 1, ...]. 依次按照batch_size的大小取出sample。  
Batch应该不需要继承与开发，如果你有特殊需求请在开发群里提出。

#### Model：用户自定的Model
必须是nn.Module的子类，  
(1) 必须实现forward方法，并且forward方法不能出现*arg这种参数. 例如  
&emsp;&emsp;&emsp;def forward(self, word_seq, *args): #这是不允许的.   
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;xxx  
返回值必须是dict的  
&emsp;&emsp;&emsp;def forward(self, word_seq, seq_lens):  
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;xxxx  
&emsp;&emsp;&emsp;return {'pred': xxx} #return的值必须是dict的。里面的预测的key推荐使用pred，但是不做强制限制。输出元素数目不限。  
(2) 如果实现了predict方法，在做evaluation的时候将调用predict方法而不是forward。如果没有predict方法，则在evaluation时调用forward方法。predict方法也不能使用*args这种参数形式，同时结果也必须返回一个dict，同样推荐key为'pred'。

#### Loss: 根据model.forward()返回的prediction(是一个dict)和batch_y计算相应的loss。  
(1) 先介绍"键映射"。 如在DataSet, Model一节所看见的那样，fastNLP并不限制Model.forward()的返回值，也不限制DataSet中target field的key。计算的loss的时候，怎么才能知道从哪里取值呢？  
这里以CrossEntropyLoss为例，一般情况下, 计算CrossEntropy需要prediction和target两个值。而在CrossEntropyLoss初始化时可以传入两个参数(pred=None, target=None), 这两个参数接受的类型是str，假设(pred='output', target='label')，那么CrossEntropyLoss会使用'output'这个key在forward的output与batch_y中寻找值;'label'也是在forward的output与batch_y中寻找值。注意这里pred或target的来源并不一定非要来自于model.forward与batch_y，也可以只来自于forward的结果。  
(2)如何创建一个自己的loss  
&emsp;&emsp;&emsp;(2.1)使用fastNLP.LossInForward, 在model.forward()的结果中包含一个为loss的key。  
&emsp;&emsp;&emsp;(2.2) trainer中使用loss(假设loss=CrossEntropyLoss())的时候其实是  
&emsp;&emsp;&emsp;&emsp;los = loss(prediction, batch_y)
        即直接调用的是loss.\__call__()方法，但是CrossEntropyLoss里面并没有自己实现\__call__方法，这是因为\__call__在LossBase中实现了。所有的loss必须继承fastNLP.core.loss.LossBase, 下面先说一下LossBase的几个方法，见下一个cell。  
(3) 尽量不要复写\__call__(), _init_param_map()方法。

###  Metric: 根据Model.forward()或者Model.predict()的结果计算metric  
metric的设计和loss的设计类似。都是传入pred_dict与target_dict进行计算。但是metric的pred_dict来源可能是Model.forward的返回值， 也可能是Model.predict(如果Model具有predict方法则会调用predict方法)的返回值，下面统一用pred_dict代替。  
(1) 这里的"键映射"与loss的"键映射"是类似的。举例来说，若Metric(pred='output', target='label')，则使用'output'到pred_dict和target_dict中寻找pred, 用'label'寻找target。   
(2) 如何创建一个自己的Metric方法  
&emsp;&emsp;Metric与loss的计算不同在于，Metric的计算有两个步骤。  
&emsp;&emsp;(2.1) <b>每个batch的输出</b>都会调用Metric的\__call__(pred_dict, target_dict)方法，而\__call__方法会调用evaluate()(需要实现)方法。   
&emsp;&emsp;(2.2) 在所有batch传入之后，调用Metric的get_metric()方法得到最终的metric值。  
&emsp;&emsp;所以Metric在调用evaluate方法时，根据拿到的数据: pred_dict与batch_y, 改变自己的状态(比如累加正确的次数，总的sample数等)。在调用get_metric()的时候给出一个最终计算结果。  
所有的Metric必须继承自fastNLP.core.metrics.MetricBase. 例子见下一个cell        
(3) 尽量不要复写\__call__(), _init_param_map()方法。


#### Tester: 用于做evaluation，应该不需要更改
重要的初始化参数有，data, model, metric  
比较重要的function是test()  
test中的运行过程   
&emsp;&emsp;predict_func = 如果有model.predict则为model.predict, 否则是model.forward  
&emsp;&emsp;for batch_x, batch_y in batch:  
&emsp;&emsp;&emsp;&emsp;# (1) 同步数据与model  
&emsp;&emsp;&emsp;&emsp;# (2) 根据predict_func的参数从batch_x中取出数据传入到predict_func中，得到结果pred_dict  
&emsp;&emsp;&emsp;&emsp;# (3) 调用metric(pred_dict, batch_y  
&emsp;&emsp;&emsp;&emsp;#(4) 当所有batch都运行完毕，会调用metric的get_metric方法，并且以返回的值作为evaluation的结果  
&emsp;&emsp;metric.get_metric()

#### Trainer: 对训练过程的封装。  
里面比较重要的function是train()  
train()中的运行过程  
&emsp;&emsp;# (1) 创建batch  
&emsp;&emsp;batch = Batch(dataset, batch_size, sampler=sampler)  
&emsp;&emsp;for batch_x, batch_y in batch:  
&emsp;&emsp;&emsp;&emsp;"""  
&emsp;&emsp;&emsp;&emsp;batch_x，batch_y都是dict。batch_x是DataSet中被设置为input的field；batch_y是DataSet中被设置为target的field。  
&emsp;&emsp;&emsp;&emsp;两个dict中的key就是DataSet中的key，value会根据情况做好padding的tensor。  
&emsp;&emsp;&emsp;&emsp;"""  
&emsp;&emsp;&emsp;&emsp;# (2)会将batch_x, batch_y中tensor移动到model所在的device  
&emsp;&emsp;&emsp;&emsp;# (3)根据model.forward的参数列表, 从batch_x中取出需要传递给forward的数据。  
&emsp;&emsp;&emsp;&emsp;# (4)获取model.forward的输出结果pred_dict，并与batch_y一起传递给loss函数, 求得loss  
&emsp;&emsp;&emsp;&emsp;# (5)对loss进行反向梯度并更新参数  
&emsp;&emsp;# (6) 如果有验证集，则需要做验证  
&emsp;&emsp;tester = Tester(model, dev_data，metric)  
&emsp;&emsp;eval_results = tester.test()  
&emsp;&emsp;# (7) 如果eval_results是当前的最佳结果，则保存模型。  